<a href="https://colab.research.google.com/github/ja390/Parallel-Computing-Assignment3/blob/main/OpenMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt-get update -qq

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
!sudo apt-get install -y openmpi-bin libopenmpi-dev > /dev/null

In [3]:
print("✔ OpenMPI Installed")
!mpirun --version

✔ OpenMPI Installed
mpirun (Open MPI) 4.1.2

Report bugs to http://www.open-mpi.org/community/help/


In [4]:
%%writefile kmeans_openmp.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <omp.h>

#define MAX_ITER 100
#define K 3  // no of clusters

typedef struct {
    double x, y;
    int cluster;
} Point;

typedef struct {
    double x, y;
} Centroid;

// Distance function
double distance(Point a, Centroid b) {
    return sqrt((a.x - b.x)*(a.x - b.x) + (a.y - b.y)*(a.y - b.y));
}

int main(int argc, char *argv[]) {

    if (argc != 2) {
        printf("Usage: %s <num_threads>\n", argv[0]);
        return 1;
    }

    int num_threads = atoi(argv[1]);
    omp_set_num_threads(num_threads);

    int n = 9; // number of points
    Point points[9] = {
        {1,2}, {2,1}, {3,3},
        {8,8}, {9,8}, {8,9},
        {4,5}, {5,6}, {6,5}
    };

    Centroid centroids[K] = {{2,2}, {8,8}, {5,5}};

    double start = omp_get_wtime();

    for (int iter = 0; iter < MAX_ITER; iter++) {

        // parallel clusters
        #pragma omp parallel for
        for (int i = 0; i < n; i++) {
            double minDist = 1e9;
            int bestCluster = 0;

            for (int j = 0; j < K; j++) {
                double dist = distance(points[i], centroids[j]);
                if (dist < minDist) {
                    minDist = dist;
                    bestCluster = j;
                }
            }

            points[i].cluster = bestCluster;
        }

        //  Parallel centroid update  Using array reductions
        double sumX[K] = {0}, sumY[K] = {0};
        int count[K] = {0};

        #pragma omp parallel for reduction(+:sumX[:K], sumY[:K], count[:K])
        for (int i = 0; i < n; i++) {
            int c = points[i].cluster;
            sumX[c] += points[i].x;
            sumY[c] += points[i].y;
            count[c]++;
        }

        // Update centroids
        for (int j = 0; j < K; j++) {
            if (count[j] > 0) {
                centroids[j].x = sumX[j] / count[j];
                centroids[j].y = sumY[j] / count[j];
            }
        }
    }

    double end = omp_get_wtime();

    printf("\n---- OpenMP K-Means Results ----\n");
    printf("Threads Used: %d\n", num_threads);
    printf("Execution Time: %f seconds\n", end - start);

    printf("\nFinal Cluster Assignments:\n");
    for (int i = 0; i < n; i++) {
        printf("Point (%.2f, %.2f) -> Cluster %d\n",
               points[i].x, points[i].y, points[i].cluster);
    }

    printf("\nFinal Centroids:\n");
    for (int j = 0; j < K; j++) {
        printf("Centroid %d: (%.2f, %.2f)\n", j, centroids[j].x, centroids[j].y);
    }

    return 0;
}


Writing kmeans_openmp.c


In [5]:
!gcc -fopenmp -O2 kmeans_openmp.c -o kmeans_openmp -lm

In [6]:
!./kmeans_openmp 1


---- OpenMP K-Means Results ----
Threads Used: 1
Execution Time: 0.000165 seconds

Final Cluster Assignments:
Point (1.00, 2.00) -> Cluster 0
Point (2.00, 1.00) -> Cluster 0
Point (3.00, 3.00) -> Cluster 0
Point (8.00, 8.00) -> Cluster 1
Point (9.00, 8.00) -> Cluster 1
Point (8.00, 9.00) -> Cluster 1
Point (4.00, 5.00) -> Cluster 2
Point (5.00, 6.00) -> Cluster 2
Point (6.00, 5.00) -> Cluster 2

Final Centroids:
Centroid 0: (2.00, 2.00)
Centroid 1: (8.33, 8.33)
Centroid 2: (5.00, 5.33)


In [7]:
!./kmeans_openmp 2


---- OpenMP K-Means Results ----
Threads Used: 2
Execution Time: 0.000222 seconds

Final Cluster Assignments:
Point (1.00, 2.00) -> Cluster 0
Point (2.00, 1.00) -> Cluster 0
Point (3.00, 3.00) -> Cluster 0
Point (8.00, 8.00) -> Cluster 1
Point (9.00, 8.00) -> Cluster 1
Point (8.00, 9.00) -> Cluster 1
Point (4.00, 5.00) -> Cluster 2
Point (5.00, 6.00) -> Cluster 2
Point (6.00, 5.00) -> Cluster 2

Final Centroids:
Centroid 0: (2.00, 2.00)
Centroid 1: (8.33, 8.33)
Centroid 2: (5.00, 5.33)


In [9]:
!./kmeans_openmp 4


---- OpenMP K-Means Results ----
Threads Used: 4
Execution Time: 0.010384 seconds

Final Cluster Assignments:
Point (1.00, 2.00) -> Cluster 0
Point (2.00, 1.00) -> Cluster 0
Point (3.00, 3.00) -> Cluster 0
Point (8.00, 8.00) -> Cluster 1
Point (9.00, 8.00) -> Cluster 1
Point (8.00, 9.00) -> Cluster 1
Point (4.00, 5.00) -> Cluster 2
Point (5.00, 6.00) -> Cluster 2
Point (6.00, 5.00) -> Cluster 2

Final Centroids:
Centroid 0: (2.00, 2.00)
Centroid 1: (8.33, 8.33)
Centroid 2: (5.00, 5.33)


In [10]:
!./kmeans_openmp 8


---- OpenMP K-Means Results ----
Threads Used: 8
Execution Time: 0.015749 seconds

Final Cluster Assignments:
Point (1.00, 2.00) -> Cluster 0
Point (2.00, 1.00) -> Cluster 0
Point (3.00, 3.00) -> Cluster 0
Point (8.00, 8.00) -> Cluster 1
Point (9.00, 8.00) -> Cluster 1
Point (8.00, 9.00) -> Cluster 1
Point (4.00, 5.00) -> Cluster 2
Point (5.00, 6.00) -> Cluster 2
Point (6.00, 5.00) -> Cluster 2

Final Centroids:
Centroid 0: (2.00, 2.00)
Centroid 1: (8.33, 8.33)
Centroid 2: (5.00, 5.33)


In [11]:
!./kmeans_openmp 16


---- OpenMP K-Means Results ----
Threads Used: 16
Execution Time: 0.021959 seconds

Final Cluster Assignments:
Point (1.00, 2.00) -> Cluster 0
Point (2.00, 1.00) -> Cluster 0
Point (3.00, 3.00) -> Cluster 0
Point (8.00, 8.00) -> Cluster 1
Point (9.00, 8.00) -> Cluster 1
Point (8.00, 9.00) -> Cluster 1
Point (4.00, 5.00) -> Cluster 2
Point (5.00, 6.00) -> Cluster 2
Point (6.00, 5.00) -> Cluster 2

Final Centroids:
Centroid 0: (2.00, 2.00)
Centroid 1: (8.33, 8.33)
Centroid 2: (5.00, 5.33)
